In [1]:
import os
import geopandas as gp
import pandas as pd
import maup
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
wd = os.getcwd()

# Boston_Precincts_CouncilDistricts_10_21_2022

## Background:
- We received a request for aggregated 2010 and 2020 Census data and population projections on City Council Districts and Precincts from the Boston City Council.
- The Council was also interested in percent change over time between 2010 and 2020.

## Approach:
- Use RDH PL datasets and population projections at the the block level.
- Query out fields which the user expressed interest in.
- Aggregate block level data to the precinct and council district levels using [maup library](https://github.com/mggg/maup)
- Create percent change fields by using the following formula: ((2020 population - 2010 population)/2010 population)*100

## Links to datasets used:
[Massachusetts block PL 94-171 2010](https://redistrictingdatahub.org/dataset/massachusetts-block-pl-94171-2010/)
[Massachusetts block boundaries (2010)](https://redistrictingdatahub.org/dataset/massachusetts-block-boundaries-2010/)
[Massachusetts Block boundaries (2020)](https://redistrictingdatahub.org/dataset/massachusetts-block-boundaries-2020/)
[Massachusetts block PL 94-171 2020](https://redistrictingdatahub.org/dataset/massachusetts-block-pl-94171-2020/)
[2021-2030 MA HastaqDNA Population Projections joined to 2020 Census Blocks, P2](https://redistrictingdatahub.org/dataset/20212030-ma-hastaqdna-population-projections-joined-to-2020-census-blocks-p2/)
[Massachusetts State boundaries (2020)](https://redistrictingdatahub.org/dataset/massachusetts-state-boundaries-2020/)
[Boston Precincts](https://bostonopendata-boston.opendata.arcgis.com/datasets/boston::boston-precinct-boundaries/explore?location=42.314086%2C-70.970025%2C11.54)
[Boston City Council Districts](https://bostonopendata-boston.opendata.arcgis.com/datasets/boston::city-council-districts-view/explore?location=42.312169%2C-71.072913%2C11.82)

For a full 'raw-from-source' file, contact info@redistrictingdatahub.org

Import all required files

In [2]:
precs = gp.read_file(os.path.join(os.path.join(wd,'Boston_Precinct_Boundaries'),'Boston_Precinct_Boundaries.shp'))
precs = precs[['DISTRICT','geometry']]
pl20  = pd.read_csv(os.path.join(os.path.join(wd,'ma_pl2020_b_csv'),'ma_pl2020_b.csv'))
b20 = gp.read_file(os.path.join(os.path.join(wd,'ma_b_2020_bound'),'ma_b_2020_bound.shp'))
b20_sub = b20[['GEOID20','geometry']]
pl10  = pd.read_csv(os.path.join(os.path.join(wd,'ma_pl2010_b'),'ma_pl2010_b.csv'))
b10 = gp.read_file(os.path.join(os.path.join(wd,'ma_2010_b_bound'),'ma_2010_b_bound.shp'))
b10_sub = b10[['GEOID','geometry']]
co_dist = gp.read_file(os.path.join(os.path.join(wd,'City_Council_Districts_View'),'city_council_districts.shp'))
co_dist = co_dist[['DISTRICT','geometry']]
proj = pd.read_csv(os.path.join(os.path.join(wd,'ma_b_proj_P2_2020tiger'),'ma_b_proj_P2_2020tiger.csv'))
st = gp.read_file(os.path.join(os.path.join(wd,'ma_st_2020_bound'),'ma_st_2020_bound.shp'))

C:\Users\SpencerNelson\AppData\Local\Temp\ipykernel_25352\1629444697.py:3: DtypeWarning: Columns (29,31,37,38,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  pl20  = pd.read_csv(os.path.join(os.path.join(wd,'ma_pl2020_b_csv'),'ma_pl2020_b.csv'))
C:\Users\SpencerNelson\AppData\Local\Temp\ipykernel_25352\1629444697.py:6: DtypeWarning: Columns (22,23,27,28,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  pl10  = pd.read_csv(os.path.join(os.path.join(wd,'ma_pl2010_b'),'ma_pl2010_b.csv'))


Query and clean PL data for 2010 and 2020

In [3]:
p2_cols_to_keep = ['P0020001','P0020002','P0020005','P0020006','P0020007','P0020008','P0020009','P0020010','P0020011']
p4_cols_to_keep = ['P0040001','P0040002','P0040005','P0040006','P0040007','P0040008','P0040009','P0040010','P0040011']
p5_cols_to_keep = ['P0050001','P0050002','P0050003','P0050004','P0050005','P0050006','P0050007','P0050008','P0050009','P0050010']
h1_cols_to_keep = ['H0010001','H0010002','H0010003']
other_cols_to_keep = ['GEOCODE']

#Make GEOCODE field for 2010
pl10['STATE']=pl10['STATE'].apply(lambda x: str(x).zfill(2))
pl10['COUNTY']=pl10['COUNTY'].apply(lambda x: str(x).zfill(3))
pl10['TRACT']=pl10['TRACT'].apply(lambda x: str(x).zfill(6))
pl10['BLOCK']=pl10['BLOCK'].apply(lambda x: str(x).zfill(4))
pl10['GEOCODE']=pl10.apply(lambda x: x['STATE']+x['COUNTY']+x['TRACT']+x['BLOCK'],axis=1)

pl10_cols = other_cols_to_keep+p2_cols_to_keep+p4_cols_to_keep+h1_cols_to_keep
pl20_cols = pl10_cols+p5_cols_to_keep

pl10_sub = pl10[pl10_cols]
pl20_sub = pl20[pl20_cols]

Rename columns for PL data 

In [4]:
pl_cols10_dict = {'GEOCODE':'GEOID','P0020001':'TOT10','P0020002':'HSP10','P0020005':'WHT_NH10','P0020006':'BLK_NH10','P0020007':'AIA_NH10','P0020008':'ASN_NH10','P0020009':'HPI_NH10','P0020010':'OTH_NH10','P0020011':'2OM_NH10',
                  'P0040001':'TOT_VAP10','P0040002':'HSP_VAP10','P0040005':'WHT_VAP10','P0040006':'BLK_VAP10','P0040007':'AIA_VAP10','P0040008':'ASN_VAP10','P0040009':'HPI_VAP10','P0040010':'OTH_VAP10','P0040011':'2OM_VAP10',
                 'H0010001':'HUNT_TOT10','H0010002':'HUNT_OCC10','H0010003':'HUNT_VAC10'}

pl_cols20_dict = {'GEOCODE':'GEOID20','P0020001':'TOT20','P0020002':'HSP20','P0020005':'WHT_NH20','P0020006':'BLK_NH20','P0020007':'AIA_NH20','P0020008':'ASN_NH20','P0020009':'HPI_NH20','P0020010':'OTH_NH20','P0020011':'2OM_NH20',
                  'P0040001':'TOT_VAP20','P0040002':'HSP_VAP20','P0040005':'WHT_VAP20','P0040006':'BLK_VAP20','P0040007':'AIA_VAP20','P0040008':'ASN_VAP20','P0040009':'HPI_VAP20','P0040010':'OTH_VAP20','P0040011':'2OM_VAP20',
                 'H0010001':'HUNT_TOT20','H0010002':'HUNT_OCC20','H0010003':'HUNT_VAC20','P0050001':'GQ_TOT20','P0050002':'GQ_INS20','P0050003':'GQ_CORR20','P0050004':'GQ_JUVE20','P0050005':'GQ_NURS20','P0050006':'GQ_OINS20','P0050007':'GQ_NINS20','P0050008':'GQ_UNIV20','P0050009':'GQ_MLTR20','P0050010':'GQ_ONINS20'}

pl20_for_comp = ['TOT20', 'HSP20', 'WHT_NH20', 'BLK_NH20', 'AIA_NH20', 'ASN_NH20', 'HPI_NH20', 'OTH_NH20', '2OM_NH20', 'TOT_VAP20', 'HSP_VAP20', 'WHT_VAP20', 'BLK_VAP20', 'AIA_VAP20', 'ASN_VAP20', 'HPI_VAP20', 'OTH_VAP20', '2OM_VAP20', 'HUNT_TOT20', 'HUNT_OCC20', 'HUNT_VAC20']
pl10_for_comp = ['TOT10', 'HSP10', 'WHT_NH10', 'BLK_NH10', 'AIA_NH10', 'ASN_NH10', 'HPI_NH10', 'OTH_NH10', '2OM_NH10', 'TOT_VAP10', 'HSP_VAP10', 'WHT_VAP10', 'BLK_VAP10', 'AIA_VAP10', 'ASN_VAP10', 'HPI_VAP10', 'OTH_VAP10', '2OM_VAP10', 'HUNT_TOT10', 'HUNT_OCC10', 'HUNT_VAC10']
comparison_dict = dict(zip(pl20_for_comp,pl10_for_comp))

pl10_sub.rename(columns = pl_cols10_dict,inplace=True)
pl20_sub.rename(columns = pl_cols20_dict,inplace=True)
display(pl10_sub.head(1))
display(pl20_sub.head(1))

,GEOID,TOT10,HSP10,WHT_NH10,BLK_NH10,AIA_NH10,ASN_NH10,HPI_NH10,OTH_NH10,2OM_NH10,...,WHT_VAP10,BLK_VAP10,AIA_VAP10,ASN_VAP10,HPI_VAP10,OTH_VAP10,2OM_VAP10,HUNT_TOT10,HUNT_OCC10,HUNT_VAC10
0,250010122001000,10,0,10,0,0,0,0,0,0,...,9,0,0,0,0,0,0,4,4,0


,GEOID20,TOT20,HSP20,WHT_NH20,BLK_NH20,AIA_NH20,ASN_NH20,HPI_NH20,OTH_NH20,2OM_NH20,...,GQ_TOT20,GQ_INS20,GQ_CORR20,GQ_JUVE20,GQ_NURS20,GQ_OINS20,GQ_NINS20,GQ_UNIV20,GQ_MLTR20,GQ_ONINS20
0,250010101001000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Subset projection data

In [5]:
proj.rename(columns = {'geoid_2020':'GEOID20'},inplace=True)
proj_fields_to_keep = []
for i in list(proj.columns):
    if '_h_' in i:
        if 'tot' in i:
            proj_fields_to_keep.append(i)
        else:
            continue
    else:
        if i!='state_fips':
            
            proj_fields_to_keep.append(i)
proj_sub = proj[proj_fields_to_keep]
proj_sub

,GEOID20,p20_nh_tot,p20_nh_wh,p20_nh_aa,p20_nh_ai,p20_nh_asi,p20_nh_pac,p20_nh_oth,p20_nh_tom,p20_h_tot,...,p29_h_tot,p30_nh_tot,p30_nh_wh,p30_nh_aa,p30_nh_ai,p30_nh_asi,p30_nh_pac,p30_nh_oth,p30_nh_tom,p30_h_tot
0,250039261001009,6,6,0,0,0,0,0,0,0,...,0,7,7,0,0,0,0,0,0,1
1,250056416002013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,250010127004025,13,10,0,0,0,0,1,2,0,...,0,10,8,0,0,0,0,0,2,0
3,250092506004006,15,15,0,0,0,0,0,0,51,...,34,16,16,0,0,0,0,0,0,32
4,250039261001015,6,6,0,0,0,0,0,0,0,...,0,7,7,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107273,250092506003008,14,10,0,0,4,0,0,0,113,...,104,14,10,0,0,4,0,0,0,103
107274,250092506003009,20,15,0,0,5,0,0,0,166,...,153,20,14,0,0,6,0,0,0,152
107275,250092506003011,14,10,0,0,4,0,0,0,114,...,106,14,10,0,0,4,0,0,0,105
107276,250092506004002,17,17,0,0,0,0,0,0,55,...,36,17,17,0,0,0,0,0,0,34


Rename projection data columns

In [6]:
proj_sub_fields = list(proj_sub.columns)
new_proj_cols = []
for i in proj_sub_fields:
    if i.startswith('p'):
        y = i.split('_')[0].replace('p','')
        h_nh = i.split('_')[1]
        race = i.split('_')[2]
        if h_nh =='h':
            h_nh = '_'
            race='HISP'
        else:
            h_nh = '_NH_'
        race_dict = {'tot':'TOT','wh':'WHT','aa':'BLK','ai':'AIA','asi':'ASN','pac':'HPI','oth':'OTH','tom':'2OM','HISP':'HISP'}
        name = 'P'+h_nh+race_dict.get(race)+y
        new_proj_cols.append(name)

proj_sub_fields.remove('GEOID20')
proj_rename_dict = dict(zip(proj_sub_fields,new_proj_cols))
proj_sub.rename(columns=proj_rename_dict,inplace=True)
proj_sub.head()

,GEOID20,P_NH_TOT20,P_NH_WHT20,P_NH_BLK20,P_NH_AIA20,P_NH_ASN20,P_NH_HPI20,P_NH_OTH20,P_NH_2OM20,P_HISP20,...,P_HISP29,P_NH_TOT30,P_NH_WHT30,P_NH_BLK30,P_NH_AIA30,P_NH_ASN30,P_NH_HPI30,P_NH_OTH30,P_NH_2OM30,P_HISP30
0,250039261001009,6,6,0,0,0,0,0,0,0,...,0,7,7,0,0,0,0,0,0,1
1,250056416002013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,250010127004025,13,10,0,0,0,0,1,2,0,...,0,10,8,0,0,0,0,0,2,0
3,250092506004006,15,15,0,0,0,0,0,0,51,...,34,16,16,0,0,0,0,0,0,32
4,250039261001015,6,6,0,0,0,0,0,0,0,...,0,7,7,0,0,0,0,0,0,1


Join projections, 2020 PL data, and shapefile together

In [7]:
proj_pl20 =pd.merge(pl20_sub,proj_sub,on='GEOID20',how='outer')
proj_pl20['GEOID20'] =proj_pl20['GEOID20'].astype(str)
ma_data20 = pd.merge(b20_sub,proj_pl20,on='GEOID20',how='outer')
ma_data20

,GEOID20,geometry,TOT20,HSP20,WHT_NH20,BLK_NH20,AIA_NH20,ASN_NH20,HPI_NH20,OTH_NH20,...,P_HISP29,P_NH_TOT30,P_NH_WHT30,P_NH_BLK30,P_NH_AIA30,P_NH_ASN30,P_NH_HPI30,P_NH_OTH30,P_NH_2OM30,P_HISP30
0,250039353003056,"POLYGON ((-73.10996 42.69902, -73.10976 42.699...",36,3,29,0,0,0,0,0,...,1,39,32,0,0,0,0,0,7,1
1,250039201021019,"POLYGON ((-73.23809 42.66205, -73.23797 42.662...",82,0,82,0,0,0,0,0,...,6,93,77,5,2,5,0,0,4,6
2,250039261001041,"POLYGON ((-73.41092 42.13063, -73.41082 42.131...",3,0,0,0,0,0,0,0,...,0,2,2,0,0,0,0,0,0,0
3,250039311002021,"POLYGON ((-73.08619 42.70946, -73.08607 42.709...",13,2,9,0,0,0,0,0,...,1,21,21,0,0,0,0,0,0,1
4,250039261001071,"POLYGON ((-73.32652 42.09883, -73.32633 42.098...",7,3,4,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107273,250173839044009,"POLYGON ((-71.44033 42.33203, -71.43995 42.332...",25,11,9,1,0,0,0,0,...,0,6,4,0,0,1,0,1,0,0
107274,250173422021016,"POLYGON ((-71.04692 42.42004, -71.04689 42.420...",101,26,55,4,0,1,0,0,...,36,75,41,21,0,3,0,5,5,37
107275,250173336011038,"POLYGON ((-71.14781 42.51086, -71.14684 42.511...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107276,250173832001013,"POLYGON ((-71.42136 42.26631, -71.42116 42.267...",280,96,80,14,1,36,0,30,...,210,53,48,2,0,3,0,0,0,214


Join 2010 PL data, and shapefile together

In [8]:
b10_sub = b10[['GEOID','geometry']]
ma_data10=pd.merge(b10_sub,pl10_sub,on='GEOID',how='outer')
ma_data10

,GEOID,geometry,TOT10,HSP10,WHT_NH10,BLK_NH10,AIA_NH10,ASN_NH10,HPI_NH10,OTH_NH10,...,WHT_VAP10,BLK_VAP10,AIA_VAP10,ASN_VAP10,HPI_VAP10,OTH_VAP10,2OM_VAP10,HUNT_TOT10,HUNT_OCC10,HUNT_VAC10
0,250010153001070,"POLYGON ((-70.29481 41.68198, -70.29519 41.682...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,250010125022055,"POLYGON ((-70.28800 41.64199, -70.28805 41.642...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,250010127002018,"POLYGON ((-70.34948 41.65485, -70.34925 41.655...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,250010125022026,"POLYGON ((-70.28962 41.64329, -70.28967 41.643...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,250010122003103,"POLYGON ((-70.34109 41.70074, -70.34094 41.700...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157503,250277441022027,"POLYGON ((-71.53773 42.14578, -71.53742 42.145...",67,2,65,0,0,0,0,0,...,59,0,0,0,0,0,0,31,31,0
157504,250277441021016,"POLYGON ((-71.53727 42.15294, -71.53723 42.152...",92,9,82,0,0,0,0,0,...,60,0,0,0,0,0,1,29,29,0
157505,250277444002012,"POLYGON ((-71.53873 42.14988, -71.53810 42.149...",42,0,37,0,0,5,0,0,...,31,0,0,2,0,0,0,14,14,0
157506,250277444002002,"POLYGON ((-71.53401 42.14346, -71.53436 42.143...",39,0,39,0,0,0,0,0,...,34,0,0,0,0,0,0,16,16,0


Modify Council District shapefile so maup can be run

In [9]:
st = st.to_crs(co_dist.crs)
erased_gdf = gp.overlay(st, co_dist, how='difference')
ma_no_co_dist = erased_gdf[['geometry']]
ma_co_dist_for_maup = gp.GeoDataFrame(pd.concat([co_dist,erased_gdf]),crs=co_dist.crs)
ma_co_dist_for_maup.reset_index(drop=True,inplace=True)
ma_co_dist_for_maup

,DISTRICT,geometry,REGION20,DIVISION20,STATEFP20,STATENS20,GEOID20,STUSPS20,NAME20,LSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20
0,1.0,"MULTIPOLYGON (((-70.99602 42.39473, -70.99600 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,"POLYGON ((-71.05593 42.36074, -71.05570 42.360...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.0,"POLYGON ((-71.08234 42.34576, -71.08094 42.344...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.0,"MULTIPOLYGON (((-71.07414 42.35769, -71.07409 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,"MULTIPOLYGON (((-71.01176 42.30895, -71.01205 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.0,"POLYGON ((-71.15858 42.25504, -71.16647 42.261...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,"POLYGON ((-71.07593 42.31245, -71.07513 42.312...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9.0,"POLYGON ((-71.12624 42.37055, -71.12573 42.369...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5.0,"POLYGON ((-71.11928 42.29022, -71.11931 42.288...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,"MULTIPOLYGON (((-71.50205 42.01699, -71.50245 ...",1,1,25,00606926,25,MA,Massachusetts,00,G4000,A,2.020440e+10,7.130654e+09,+42.1565196,-071.4895915


Create list of variables to aggregate for 2020 data and aggregate to council districts

In [10]:
vars20 = list(ma_data20.columns)
vars20.remove('GEOID20')
vars20.remove('geometry')
ma_data20 = ma_data20.to_crs(ma_co_dist_for_maup.crs)

co_20_assign = maup.assign(ma_data20,ma_co_dist_for_maup)
ma_co_dist_for_maup[vars20] = ma_data20[vars20].groupby(co_20_assign).sum()
ma_co_dist_for_maup

C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\indexed_geometries.py:15: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometry.index = i
C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


,DISTRICT,geometry,REGION20,DIVISION20,STATEFP20,STATENS20,GEOID20,STUSPS20,NAME20,LSAD20,...,P_HISP29,P_NH_TOT30,P_NH_WHT30,P_NH_BLK30,P_NH_AIA30,P_NH_ASN30,P_NH_HPI30,P_NH_OTH30,P_NH_2OM30,P_HISP30
0,1.0,"MULTIPOLYGON (((-70.99602 42.39473, -70.99600 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37530,53101,43605,2425,36,4545,106,199,2185,38352
1,2.0,"POLYGON ((-71.05593 42.36074, -71.05570 42.360...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10293,82807,61347,4896,88,13521,83,215,2657,10554
2,7.0,"POLYGON ((-71.08234 42.34576, -71.08094 42.344...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23233,57432,16826,31955,372,5268,122,705,2184,23799
3,8.0,"MULTIPOLYGON (((-71.07414 42.35769, -71.07409 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12049,66831,46653,4687,118,12090,51,174,3058,12387
4,3.0,"MULTIPOLYGON (((-71.01176 42.30895, -71.01205 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14471,62482,27662,19748,141,11072,0,1045,2814,14826
5,6.0,"POLYGON ((-71.15858 42.25504, -71.16647 42.261...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16811,73482,54690,8839,13,6116,0,281,3543,17175
6,4.0,"POLYGON ((-71.07593 42.31245, -71.07513 42.312...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23071,64748,7354,49612,168,4492,0,654,2468,23635
7,9.0,"POLYGON ((-71.12624 42.37055, -71.12573 42.369...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10964,62344,43062,3389,150,12826,53,908,1956,11228
8,5.0,"POLYGON ((-71.11928 42.29022, -71.11931 42.288...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25458,73799,23723,45226,111,1711,0,376,2652,26087
9,NaN,"MULTIPOLYGON (((-71.50205 42.01699, -71.50245 ...",1,1,25,00606926,25,MA,Massachusetts,00,...,854841,5667896,4665668,350669,7121,449922,1831,47324,145361,874725


Create list of variables to aggregate for 2020 data and aggregate to council districts

In [11]:
vars10 = list(ma_data10.columns)
vars10.remove('GEOID')
vars10.remove('geometry')
ma_data10 = ma_data10.to_crs(ma_co_dist_for_maup.crs)

co_10_assign = maup.assign(ma_data10,ma_co_dist_for_maup)
ma_co_dist_for_maup[vars10] = ma_data10[vars10].groupby(co_10_assign).sum()
ma_co_dist_for_maup = ma_co_dist_for_maup[~ma_co_dist_for_maup['DISTRICT'].isna()]
ma_co_dist_for_maup

C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\indexed_geometries.py:15: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometry.index = i
C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


,DISTRICT,geometry,REGION20,DIVISION20,STATEFP20,STATENS20,GEOID20,STUSPS20,NAME20,LSAD20,...,WHT_VAP10,BLK_VAP10,AIA_VAP10,ASN_VAP10,HPI_VAP10,OTH_VAP10,2OM_VAP10,HUNT_TOT10,HUNT_OCC10,HUNT_VAC10
0,1.0,"MULTIPOLYGON (((-70.99602 42.39473, -70.99600 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33258,1607,74,2460,15,596,640,31431,29015,2416
1,2.0,"POLYGON ((-71.05593 42.36074, -71.05570 42.360...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,44268,2968,75,8968,20,195,760,37695,33923,3772
2,7.0,"POLYGON ((-71.08234 42.34576, -71.08094 42.344...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15306,20378,174,2828,18,1073,1450,26051,24225,1826
3,8.0,"MULTIPOLYGON (((-71.07414 42.35769, -71.07409 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45921,3740,95,9897,20,200,1519,32784,30182,2602
4,3.0,"MULTIPOLYGON (((-71.01176 42.30895, -71.01205 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21930,11859,112,7202,24,2890,1771,27279,25203,2076
5,6.0,"POLYGON ((-71.15858 42.25504, -71.16647 42.261...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38974,5700,90,3393,11,244,848,31537,29931,1606
6,4.0,"POLYGON ((-71.07593 42.31245, -71.07513 42.312...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4735,29783,154,2087,7,1123,1364,25324,22976,2348
7,9.0,"POLYGON ((-71.12624 42.37055, -71.12573 42.369...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45094,2793,57,9949,24,911,1456,31912,30610,1302
8,5.0,"POLYGON ((-71.11928 42.29022, -71.11931 42.288...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16919,24814,121,952,18,331,932,28478,26644,1834


Function for calculating percent increase

In [12]:
def calculate_per(val20, val10):
    try:
        new_val = round((((val20-val10)/val10)*100),2)
    except:
        new_val = 0.00
    return new_val

Calculate percent change for council districts

In [13]:
for k,v in comparison_dict.items():
    if k.startswith('TOT'):
        pre = 'TOT'
    elif k.startswith('HSP'):
        pre='HSP'.replace('20','')
    elif k.startswith('HUNT'):
        pre='H'+k.split('_')[1].replace('20','')
    else:
        pre = k.split('_')[0]
    if '_V' in k:
        if k.startswith('H'):
            vap = ''
        else:
            vap = 'V'
    else:
        vap=''
    new_col_name = pre+vap+'_P1020'
    ma_co_dist_for_maup[new_col_name]= ma_co_dist_for_maup.apply(lambda x: calculate_per(x[k],x[v]),axis=1)
#Reorganize columns
co_dist_cols = list(ma_co_dist_for_maup.columns)
co_dist_cols.remove('geometry')
co_dist_cols.append('geometry')
council_districts = ma_co_dist_for_maup[co_dist_cols]
council_districts

,DISTRICT,REGION20,DIVISION20,STATEFP20,STATENS20,GEOID20,STUSPS20,NAME20,LSAD20,MTFCC20,...,WHTV_P1020,BLKV_P1020,AIAV_P1020,ASNV_P1020,OTHV_P1020,2OMV_P1020,HTOT_P1020,HOCC_P1020,HVAC_P1020,geometry
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.12,22.71,-36.49,47.36,3.36,197.50,11.54,12.10,4.76,"MULTIPOLYGON (((-70.99602 42.39473, -70.99600 ..."
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.31,19.41,-22.67,32.74,111.28,169.74,30.21,28.90,41.97,"POLYGON ((-71.05593 42.36074, -71.05570 42.360..."
2,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.70,-3.96,-46.55,109.65,-5.03,144.69,8.85,7.82,22.56,"POLYGON ((-71.08234 42.34576, -71.08094 42.344..."
3,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.28,7.38,-29.47,46.36,149.00,68.07,7.38,5.70,26.83,"MULTIPOLYGON (((-71.07414 42.35769, -71.07409 ..."
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.99,-16.48,-11.61,28.62,-36.61,124.68,7.45,8.64,-6.98,"MULTIPOLYGON (((-71.01176 42.30895, -71.01205 ..."
5,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.58,2.82,-27.78,58.47,70.49,142.33,7.70,7.08,19.18,"POLYGON ((-71.15858 42.25504, -71.16647 42.261..."
6,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.80,-6.26,20.13,42.02,-0.18,168.40,4.95,8.33,-28.15,"POLYGON ((-71.07593 42.31245, -71.07513 42.312..."
7,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.96,9.52,31.58,35.38,-8.45,80.77,7.68,6.25,41.24,"POLYGON ((-71.12624 42.37055, -71.12573 42.369..."
8,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.67,6.53,-13.22,38.13,54.08,148.18,4.55,6.46,-23.06,"POLYGON ((-71.11928 42.29022, -71.11931 42.288..."


Modify Precinct shapefile so maup can be run

In [14]:
st = st.to_crs(precs.crs)
erased_gdf = gp.overlay(st, precs, how='difference')
ma_no_prec = erased_gdf[['geometry']]
ma_prec_for_maup = gp.GeoDataFrame(pd.concat([precs,ma_no_prec]),crs=precs.crs)
ma_prec_for_maup.reset_index(drop=True,inplace=True)
ma_prec_for_maup

,DISTRICT,geometry
0,01-01,"POLYGON ((-70.97289 42.35361, -70.97509 42.343..."
1,01-02,"POLYGON ((-71.03523 42.36774, -71.03624 42.368..."
2,01-03,"POLYGON ((-71.03523 42.36774, -71.03538 42.367..."
3,01-04,"POLYGON ((-71.04611 42.37481, -71.04169 42.374..."
4,01-05,"POLYGON ((-71.03698 42.37397, -71.03691 42.374..."
...,...,...
271,22-10,"POLYGON ((-71.16488 42.34643, -71.16516 42.346..."
272,22-11,"POLYGON ((-71.16682 42.35608, -71.16677 42.356..."
273,22-12,"POLYGON ((-71.14760 42.35888, -71.14758 42.358..."
274,22-13,"POLYGON ((-71.16761 42.35061, -71.16776 42.350..."


Aggregate 2020 data to council districts

In [15]:
ma_data20 = ma_data20.to_crs(ma_prec_for_maup.crs)
prec_20_assign = maup.assign(ma_data20,ma_prec_for_maup)
ma_prec_for_maup[vars20] = ma_data20[vars20].groupby(prec_20_assign).sum()
ma_prec_for_maup

C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


,DISTRICT,geometry,TOT20,HSP20,WHT_NH20,BLK_NH20,AIA_NH20,ASN_NH20,HPI_NH20,OTH_NH20,...,P_HISP29,P_NH_TOT30,P_NH_WHT30,P_NH_BLK30,P_NH_AIA30,P_NH_ASN30,P_NH_HPI30,P_NH_OTH30,P_NH_2OM30,P_HISP30
0,01-01,"POLYGON ((-70.97289 42.35361, -70.97509 42.343...",2340,652,1420,60,1,70,0,40,...,989,2341,2060,57,18,100,0,2,104,1011
1,01-02,"POLYGON ((-71.03523 42.36774, -71.03624 42.368...",2603,1303,994,42,0,91,0,32,...,2086,1829,1438,113,0,152,30,0,96,2133
2,01-03,"POLYGON ((-71.03523 42.36774, -71.03538 42.367...",5111,1804,2287,274,4,449,2,78,...,2411,2452,1741,293,6,324,0,5,83,2444
3,01-04,"POLYGON ((-71.04611 42.37481, -71.04169 42.374...",3019,1494,1113,122,6,159,0,40,...,2089,1369,1160,123,0,33,0,0,53,2140
4,01-05,"POLYGON ((-71.03698 42.37397, -71.03691 42.374...",3539,1958,1124,79,7,145,1,106,...,3927,1430,1180,24,0,54,1,0,171,4025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,22-10,"POLYGON ((-71.16488 42.34643, -71.16516 42.346...",2279,201,1633,86,0,233,0,41,...,356,2782,1941,197,0,407,18,22,197,368
272,22-11,"POLYGON ((-71.16682 42.35608, -71.16677 42.356...",1663,144,1096,53,0,247,1,32,...,326,2047,1487,93,0,306,0,93,68,338
273,22-12,"POLYGON ((-71.14760 42.35888, -71.14758 42.358...",2206,576,926,189,0,317,0,94,...,1186,1903,1517,59,0,308,0,0,19,1233
274,22-13,"POLYGON ((-71.16761 42.35061, -71.16776 42.350...",1513,140,1138,37,0,141,0,15,...,71,1726,1352,176,0,123,0,0,75,76


Aggregate 2010 data to council districts

In [16]:
ma_data10 = ma_data10.to_crs(ma_prec_for_maup.crs)
prec_10_assign = maup.assign(ma_data10,ma_prec_for_maup)
ma_prec_for_maup[vars10] = ma_data10[vars10].groupby(prec_10_assign).sum()
ma_prec_for_maup = ma_prec_for_maup[~ma_prec_for_maup['DISTRICT'].isna()]
ma_prec_for_maup

C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


,DISTRICT,geometry,TOT20,HSP20,WHT_NH20,BLK_NH20,AIA_NH20,ASN_NH20,HPI_NH20,OTH_NH20,...,WHT_VAP10,BLK_VAP10,AIA_VAP10,ASN_VAP10,HPI_VAP10,OTH_VAP10,2OM_VAP10,HUNT_TOT10,HUNT_OCC10,HUNT_VAC10
0,01-01,"POLYGON ((-70.97289 42.35361, -70.97509 42.343...",2340,652,1420,60,1,70,0,40,...,1074,38,3,45,0,27,27,1002,890,112
1,01-02,"POLYGON ((-71.03523 42.36774, -71.03624 42.368...",2603,1303,994,42,0,91,0,32,...,828,40,2,42,0,53,33,1017,937,80
2,01-03,"POLYGON ((-71.03523 42.36774, -71.03538 42.367...",5111,1804,2287,274,4,449,2,78,...,1275,212,5,227,0,57,89,1840,1721,119
3,01-04,"POLYGON ((-71.04611 42.37481, -71.04169 42.374...",3019,1494,1113,122,6,159,0,40,...,512,43,6,40,0,59,24,919,849,70
4,01-05,"POLYGON ((-71.03698 42.37397, -71.03691 42.374...",3539,1958,1124,79,7,145,1,106,...,969,47,1,74,0,93,40,1386,1281,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,22-09,"POLYGON ((-71.15529 42.34035, -71.15554 42.340...",2347,120,1636,51,0,426,0,39,...,1498,37,0,319,2,11,52,986,935,51
271,22-10,"POLYGON ((-71.16488 42.34643, -71.16516 42.346...",2279,201,1633,86,0,233,0,41,...,1842,81,1,248,0,34,29,1140,1103,37
272,22-11,"POLYGON ((-71.16682 42.35608, -71.16677 42.356...",1663,144,1096,53,0,247,1,32,...,949,64,1,152,0,12,28,629,600,29
273,22-12,"POLYGON ((-71.14760 42.35888, -71.14758 42.358...",2206,576,926,189,0,317,0,94,...,903,138,5,242,0,29,30,792,772,20


Calculate percent change for council districts

In [17]:
for k,v in comparison_dict.items():
    if k.startswith('TOT'):
        pre = 'TOT'
    elif k.startswith('HSP'):
        pre='HSP'
    elif k.startswith('HUNT'):
        pre='H'+k.split('_')[1].replace('20','')
    else:
        pre = k.split('_')[0]
    if '_V' in k:
        if k.startswith('HUNT'):
            vap = ''
        else:
            vap = 'V'
    else:
        vap=''
    new_col_name = pre+vap+'_P1020'
    ma_prec_for_maup[new_col_name]= ma_prec_for_maup.apply(lambda x: calculate_per(x[k],x[v]),axis=1)

#Reorganize columns
prec_cols = list(ma_prec_for_maup.columns)
prec_cols.remove('geometry')
prec_cols.append('geometry')
precincts = ma_prec_for_maup[prec_cols]
precincts

,DISTRICT,TOT20,HSP20,WHT_NH20,BLK_NH20,AIA_NH20,ASN_NH20,HPI_NH20,OTH_NH20,2OM_NH20,...,BLKV_P1020,AIAV_P1020,ASNV_P1020,HPIV_P1020,OTHV_P1020,2OMV_P1020,HTOT_P1020,HOCC_P1020,HVAC_P1020,geometry
0,01-01,2340,652,1420,60,1,70,0,40,97,...,18.42,-66.67,51.11,0.0,14.81,188.89,11.88,15.28,-15.18,"POLYGON ((-70.97289 42.35361, -70.97509 42.343..."
1,01-02,2603,1303,994,42,0,91,0,32,141,...,2.50,-100.00,92.86,0.0,-50.94,206.06,6.49,8.00,-11.25,"POLYGON ((-71.03523 42.36774, -71.03624 42.368..."
2,01-03,5111,1804,2287,274,4,449,2,78,213,...,13.21,-20.00,87.67,0.0,19.30,86.52,52.99,49.68,100.84,"POLYGON ((-71.03523 42.36774, -71.03538 42.367..."
3,01-04,3019,1494,1113,122,6,159,0,40,85,...,113.95,-50.00,252.50,0.0,-47.46,200.00,56.15,53.83,84.29,"POLYGON ((-71.04611 42.37481, -71.04169 42.374..."
4,01-05,3539,1958,1124,79,7,145,1,106,119,...,40.43,400.00,68.92,0.0,-30.11,147.50,2.31,2.65,-1.90,"POLYGON ((-71.03698 42.37397, -71.03691 42.374..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,22-09,2347,120,1636,51,0,426,0,39,75,...,37.84,0.00,16.93,-100.0,127.27,17.31,8.62,8.56,9.80,"POLYGON ((-71.15529 42.34035, -71.15554 42.340..."
271,22-10,2279,201,1633,86,0,233,0,41,85,...,-2.47,-100.00,-14.52,0.0,-29.41,124.14,-10.44,-10.70,-2.70,"POLYGON ((-71.16488 42.34643, -71.16516 42.346..."
272,22-11,1663,144,1096,53,0,247,1,32,90,...,-21.88,-100.00,33.55,0.0,116.67,167.86,-0.95,-0.33,-13.79,"POLYGON ((-71.16682 42.35608, -71.16677 42.356..."
273,22-12,2206,576,926,189,0,317,0,94,104,...,4.35,-100.00,11.16,0.0,168.97,130.00,3.79,2.46,55.00,"POLYGON ((-71.14760 42.35888, -71.14758 42.358..."


Extract data

In [18]:
precincts.to_file('./ma_boston_precincts_2010_2020_demographic_change.shp')
council_districts.to_file('./ma_boston_council_districts_2010_2020_demographic_change.shp')
council_districts.drop(columns='geometry',inplace=True)
precincts.drop(columns = 'geometry',inplace=True)
precincts.to_csv('./ma_boston_precincts_2010_2020_demographic_change.csv',index=False)
council_districts.to_csv('./ma_boston_council_districts_2010_2020_demographic_change.csv',index=False)